# №8 Дәріс: ОББ тәжірибесі: Класстарды жобалау және іске асыру

### Дәріс мақсаттары:
1.  **Инкапсуляцияны тереңірек түсіну:** Геттерлер, сеттерлер және қасиеттер (properties) арқылы атрибуттарға қол жеткізуді басқаруды үйрену.
2.  **Негізгі "магиялық" әдістерді зерттеу:** `__init__`, `__str__` және `__del__` әдістерінің мақсатын түсіну.
3.  **Класстарды жобалауды үйрену:** Идеядан бастап демонстрациялық бағдарламаға дейінгі классты құрудың толық циклін қарастыру.
4.  **Объектілер жинағымен жұмыс істеуді меңгеру:** Объектілерді тізімдерде сақтауды және олармен өзара әрекеттесуді түсіну.

## 1-бөлім. Қысқаша қайталау

Өткен дәрісте біз мыналарды білдік:
- **Класс** — бұл сызба.
- **Объект** — бұл сызба бойынша жасалған дана.
- `__init__(self, ...)` — бұл объект құрылған кезде оның атрибуттарын инициализациялайтын **конструктор**.
- `self` — бұл класс ішіндегі объектінің өзіне сілтеме.
- **Атрибуттар** — бұл объектінің деректері (айнымалылар).
- **Әдістер** — бұл объектінің мінез-құлқы (функциялар).

Бүгін біз класстарды құрудың практикалық аспектілеріне тереңірек үңілеміз.

## 2-бөлім. Инкапсуляция тәжірибеде: Геттерлер мен Сеттерлер

Инкапсуляция — бұл тек деректер мен әдістерді біріктіру ғана емес, сонымен қатар бұл деректерді қате өзгерістерден қорғау. Интернет-дүкендегі тауарға арналған классты қарастырайық.

In [ ]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self.price = price

book = Product("Соғыс және бейбітшілік", 5000)
print(f"{book.name} {book.price} тг. тұрады.")

# Мәселе: кез келген адам қате мән меншіктей алады
book.price = -1000
print(f"Жаңа бағасы: {book.price} тг. -> Бұл мағынасыз!")

Бұл мәселені шешу үшін біз атрибутты жасырып, онымен жұмыс істеуге арналған арнайы әдістерді ұсынуымыз керек.

- **Атау келісімі:** Сырттан тікелей қол жеткізуге арналмаған атрибуттың басына бір астын сызу қойылады (мысалы, `_price`). Бұл "қорғалған" атрибут.
- **Сеттер (setter):** Атрибуттың мәнін **орнатуға** арналған әдіс. Оның ішінде біз тексеру (валидация) логикасын қоса аламыз.
- **Геттер (getter):** Атрибуттың мәнін **алуға** арналған әдіс.

In [ ]:
class ProductWithValidation:
    def __init__(self, name, price):
        self.name = name
        self._price = price # Қорғалған атрибутты қолданамыз

    # Бағаға арналған геттер
    def get_price(self):
        return self._price

    # Бағаға арналған сеттер
    def set_price(self, new_price):
        if new_price > 0:
            self._price = new_price
            print("Баға сәтті жаңартылды.")
        else:
            print("Қате: баға оң сан болуы керек!")

phone = ProductWithValidation("Смартфон X", 150000)

# Бағаны геттер арқылы аламыз
print(f"Ағымдағы баға: {phone.get_price()}")

# Сеттер арқылы қате баға орнатуға тырысамыз
phone.set_price(-5000)

# Дұрыс баға орнатамыз
phone.set_price(145000)
print(f"Жаңа баға: {phone.get_price()}")

### 2.1-бөлім. Қол жеткізу деңгейлері және @property декораторлары

Python-да Java немесе C++ сияқты қатаң `private` немесе `protected` кілт сөздері жоқ. Қол жеткізуді басқару **атау келісімдеріне** негізделген.

1.  `self.price` — **Public (Жариялы)**. Атрибут кез келген жерден оқуға және жазуға қолжетімді. Бұл стандартты мінез-құлық.
2.  `self._price` — **Protected (Қорғалған)**. Басындағы бір астын сызу — басқа бағдарламашыларға арналған **сигнал**: "Бұл атрибут класстың ішкі бөлігі болып табылады. Оны пайдалана аласыз, бірақ егер өз әрекеттеріңізге сенімді болмасаңыз, оны тікелей өзгертпегеніңіз жөн". Техникалық тұрғыдан оған қол жеткізу шектелмеген.
3.  `self.__price` — **Private (Жеке)**. Басындағы екі астын сызу **аттарды бұрмалау (name mangling)** механизмін іске қосады. Python бұл атрибутты автоматты түрде `_КлассАты__price` деп өзгертеді. Бұл оған сырттан кездейсоқ қол жеткізуді іс жүзінде мүмкін етпейді, осылайша сенімдірек инкапсуляцияны қамтамасыз етеді.

In [ ]:
class TestAccess:
    def __init__(self):
        self.public = "Мен жариялымын"
        self._protected = "Мен қорғалғанмын"
        self.__private = "Мен жекемін"

t = TestAccess()
print(t.public)
print(t._protected)

try:
    print(t.__private) # Бұл қате тудырады!
except AttributeError as e:
    print(f"Қате: {e}")

# Бірақ бұрмаланған атты білсек, бәрібір қол жеткізе аламыз
print(f"Бұрмаланған ат арқылы қол жеткізу: {t._TestAccess__private}")

### Python стиліндегі тәсіл: `@property` және `@setter` декораторлары

`get_price()` және `set_price()` әдістерін шақыру функционалды, бірақ өте ыңғайлы емес. `product.price` қарапайымдылығын сақтап, бірақ валидациямен болғанын қалаймыз. Ол үшін Python-да **декораторлар** бар.

-   `@property` — әдісті жай атрибут сияқты көрінетін (жақшасыз шақырылатын) **геттерге** айналдырады.
-   `@<аты>.setter` — әдісті осы "атрибутқа" мән беруге тырысқанда шақырылатын **сеттерге** айналдырады.

In [ ]:
class ProductPythonic:
    def __init__(self, name, price):
        self.name = name
        self.__price = price # Сақтау үшін жеке атрибутты қолданамыз

    @property
    def price(self):
        """Бұл геттер. `obj.price` оқығанда шақырылады."""
        print("(Геттер шақырылды)")
        return self.__price

    @price.setter
    def price(self, new_price):
        """Бұл сеттер. `obj.price = value` жазғанда шақырылады."""
        print("(Сеттер шақырылды)")
        if new_price > 0:
            self.__price = new_price
        else:
            print("Қате: баға оң сан болуы керек!")

# --- Жұмысын көрсету ---
tv = ProductPythonic("Теледидар 4K", 250000)

# Мәнді оқимыз. @property-мен белгіленген әдіс шақырылады
current_price = tv.price 
print(f"Ағымдағы баға: {current_price}")

# Мән беруге тырысамыз. @price.setter-мен белгіленген әдіс шақырылады
print("\nҚате баға орнату әрекеті...")
tv.price = -100

print("\nДұрыс баға орнату әрекеті...")
tv.price = 240000
print(f"Жаңа баға: {tv.price}")

## 3-бөлім. "Магиялық" әдістер (Dunder Methods)

Басы мен соңында екі астын сызуы бар әдістер (мысалы, `__init__`) "магиялық" немесе dunder-әдістер (Double Underscore сөзінен) деп аталады. Олар объектінің өмірлік циклінде арнайы функцияларды орындайды.

### `__init__` — Конструктор
Біз онымен таныспыз. Объектіні инициализациялау үшін оны **құру сәтінде** шақырылады.

### `__str__` — Объектінің жолдық көрінісі

Егер біз `phone` объектісін басып шығаруға тырыссақ не болады?

In [ ]:
print(phone)

Біз ақпаратсыз хабарлама көреміз. `__str__` әдісі объектіні `print()` арқылы басып шығарғанда немесе `str()` арқылы жолға түрлендіргенде оның қалай көрінетінін анықтауға мүмкіндік береді.

In [ ]:
class ProductWithStr:
    def __init__(self, name, price):
        self.name = name
        self._price = price
    
    # ... (геттерлер мен сеттерлер қысқалық үшін алынып тасталды) ...
    
    # Жолдық көріністі анықтаймыз
    def __str__(self):
        # Бұл әдіс МІНДЕТТІ түрде жолды қайтаруы керек
        return f"Тауар: {self.name}, Бағасы: {self._price} тг."

laptop = ProductWithStr("Ноутбук Pro", 550000)
print(laptop) # Енді шығару әдемі және ақпаратты болады

### `__del__` — Деструктор

Деструктор — бұл объектіні Python-ның қоқыс жинағышы **жою алдында** шақырылатын әдіс. Ол сирек қолданылады, негізінен сыртқы ресурстарды босату үшін (мысалы, объект ашқан файлды немесе желілік байланысты жабу).

> **Маңызды:** Python-да `__del__` қашан шақырылатынын нақты басқара алмайсыз. Бұл қоқыс жинағышқа байланысты. Сондықтан оған маңызды логика үшін сенім артпау керек.

In [ ]:
class TemporaryObject:
    def __init__(self, name):
        self.name = name
        print(f"'{self.name}' объектісі құрылды.")
    
    def __del__(self):
        print(f"'{self.name}' объектісі жойылуда! Қош бол, әлем!")

def create_temp_object():
    print("--- Функцияға кіреміз ---")
    temp = TemporaryObject("Уақытша")
    print("--- Функциядан шығамыз ---")

create_temp_object()
# Функциядан шыққаннан кейін, 'temp' объектісі қолжетімсіз болады,
# және қоқыс жинағыш оны жояды, __del__ шақырылады

## 4-бөлім. Бірнеше объектімен жұмыс

ОББ-ның күші бір типтегі көптеген объектілермен жұмыс істегенде ашылады. Оларды тізімде немесе сөздікте сақтау ыңғайлы.

**Мәселе:** Тауарлар каталогын құру, ең қымбат тауарды табу және қоймадағы барлық тауарлардың жалпы құнын есептеу.

In [ ]:
# ProductWithStr класын қолданамыз
catalog = [
    ProductWithStr("Ноутбук Pro", 550000),
    ProductWithStr("Смартфон X", 150000),
    ProductWithStr("Құлаққап Air", 85000)
]

print("--- Тауарлар каталогы ---")
for product in catalog:
    print(product) # __str__ қолданылады

# --- Каталогты талдау ---
total_cost = 0
most_expensive_product = None

for product in catalog:
    total_cost += product._price # Қарапайымдылық үшін тікелей қол жеткіземіз
    
    if most_expensive_product is None or product._price > most_expensive_product._price:
        most_expensive_product = product

print(f"\nТауарлардың жалпы құны: {total_cost} тг.")
print(f"Ең қымбат тауар: {most_expensive_product}")

## Қорытынды

Бұл дәрісте біз теориядан практикаға көштік:
1.  **Геттерлер мен сеттерлер**, сондай-ақ олардың `@property` түріндегі талғампаз баламасы арқылы деректерді қорғауды үйрендік.
2.  Класстарымызды ыңғайлы және болжамды ететін **магиялық әдістерді** (`__str__`, `__del__`) талдадық.
3.  Бізге таныс циклдерді қолдана отырып, **объектілер тізімдерімен** тиімді жұмыс істеуге болатынын көрдік.

Енді сіздер берілген спецификация бойынша класстарды өз бетінше жобалап, іске асыруға арналған зертханалық жұмысты орындауға дайынсыздар.